In [ ]:
%load_ext autoreload
%autoreload 2
from rich import print # noqa
from rich.table import Table # noqa

from imgtools.crawler.crawl2 import (
    crawl_directory, # noqa
    find_dicoms,
    json, # noqa
    pathlib,
)
from pydicom import dcmread # noqa
from imgtools.crawler.parse_dicom import parse_dicom
from imgtools.dicom.input import (
    SEGRefSeries,
    SEGRefSOPs,
    SR_RefSeries,
    SR_RefSOPs,
    RTDOSERefPlanSOP,
    RTDOSERefStructSOP,
    RTPLANRefStructSOP,
    rtdose_reference_uids,
    rtplan_reference_uids,
    rtstruct_reference_uids,
    seg_reference_uids,
    sr_reference_uids,
)
from imgtools.logging import logger # noqa
from imgtools.utils.timer import timer # noqa

logger.setLevel("DEBUG")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
testpath = pathlib.Path().cwd().parent / "testdata"
dicoms = find_dicoms(testpath, recursive=True, check_header=False, extension="dcm")
print(f'Found {len(dicoms)} dicoms in {testpath}')

2025-02-27T16:33:22-0500 [debug    ] Looking for DICOM files        [imgtools] call=utils.find_dicoms:161 check_header=False directory=PosixPath('/home/gpudual/bhklab/radiomics/Projects/med-imagetools/testdata') limit=None recursive=True search_input=None search_pattern=*.dcm


Found 84255 dicoms in /home/gpudual/bhklab/radiomics/Projects/med-imagetools/testdata

In [4]:
# filter groups by looking if the path has '{MODALITY}_Series' in it
from collections import defaultdict

groups = defaultdict(list)

for modality in ["CT", "MR", "PT", "SEG", "RTSTRUCT", "RTDOSE", "RTPLAN", "SR"]:
    groups[modality] = [d for d in dicoms if f"{modality}_Series" in d.as_posix()]

# print a summary of the groups
table = Table(title="Groups")
table.add_column("Modality")
table.add_column("Count")
for modality, group in groups.items():
    table.add_row(modality, str(len(group)))
print(table)

       Groups       
┏━━━━━━━━━━┳━━━━━━━┓
┃ Modality ┃ Count ┃
┡━━━━━━━━━━╇━━━━━━━┩
│ CT       │ 60317 │
│ MR       │ 19772 │
│ PT       │ 3848  │
│ SEG      │ 44    │
│ RTSTRUCT │ 296   │
│ RTDOSE   │ 8     │
│ RTPLAN   │ 7     │
│ SR       │ 227   │
└──────────┴───────┘

In [7]:
rtplan_files = groups["RTPLAN"]

# Create a table to list the number of UIDs
uid_table = Table(title="RTPLAN Reference UIDs")
uid_table.add_column("RTPLAN File")
uid_table.add_column("Number of Referenced Instance UIDs")

for rtplan in rtplan_files:
    uids = rtplan_reference_uids(rtplan)
    uid_table.add_row(str(rtplan.relative_to(testpath.parent)), '1')

print(uid_table)

                                    RTPLAN Reference UIDs                                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ RTPLAN File                                          ┃ Number of Referenced Instance UIDs ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ testdata/HN-HGJ-072/RTPLAN_Series-58071/1-1.dcm      │ 1                                  │
│ testdata/VS-SEG-002/RTPLAN_Series-22208/00000001.dcm │ 1                                  │
│ testdata/VS-SEG-002/RTPLAN_Series-04388/00000001.dcm │ 1                                  │
│ testdata/VS-SEG-001/RTPLAN_Series-69252/00000001.dcm │ 1                                  │
│ testdata/VS-SEG-001/RTPLAN_Series-25725/00000001.dcm │ 1                                  │
│ testdata/HN-CHUS-082/RTPLAN_Series-37374/1-1.dcm     │ 1                                  │
│ testdata/HN-CHUS-052/RTPLAN_Series-04314/1-1.dcm     │ 1                                  │
└──────────────────────────────────────────────────────┴────────────────────────────────────┘

In [8]:
seg_files = groups["SEG"]

seg_table = Table(title="SEG Reference UIDs")
seg_table.add_column("SEG File")
seg_table.add_column("# Ref Series UIDs")
seg_table.add_column("# Ref SOP UIDs")

for seg in seg_files:
    match seg_reference_uids(seg):
        case SEGRefSeries(ref_uid), SEGRefSOPs(ref_sops):
            seg_table.add_row(str(seg.relative_to(testpath.parent)), "1", str(len(ref_sops)))
        case SEGRefSOPs(ref_sops):
            seg_table.add_row(str(seg.relative_to(testpath.parent)), "0", str(len(ref_sops)))

print(seg_table)

                                         SEG Reference UIDs                                         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ SEG File                                                    ┃ # Ref Series UIDs ┃ # Ref SOP UIDs ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ testdata/Adrenal_Ki67_Seg_002/SEG_Series-67488/00000001.dcm │ 1                 │ 56             │
│ testdata/PCAMPMRI-00001/SEG_Series-9.698/00000001.dcm       │ 1                 │ 30             │
│ testdata/PCAMPMRI-00001/SEG_Series-8.345/00000001.dcm       │ 1                 │ 20             │
│ testdata/PCAMPMRI-00001/SEG_Series-6.412/00000001.dcm       │ 1                 │ 16             │
│ testdata/PCAMPMRI-00001/SEG_Series-7.172/00000001.dcm       │ 1                 │ 30             │
│ testdata/PCAMPMRI-00001/SEG_Series-119.3/00000001.dcm       │ 1                 │ 14             │
│ testdata/PCAMPMRI-00001/SEG_Series-0.881/00000001.dcm       │ 1                 │ 20             │
│ testdata/LIDC-IDRI-0340/SEG_Series-50648/00000001.dcm       │ 1                 │ 44             │
│ testdata/LIDC-IDRI-0340/SEG_Series-47699/00000001.dcm       │ 1                 │ 14             │
│ testdata/LIDC-IDRI-0340/SEG_Series-30745/00000001.dcm       │ 1                 │ 42             │
│ testdata/LIDC-IDRI-0340/SEG_Series-93006/00000001.dcm       │ 1                 │ 11             │
│ testdata/LIDC-IDRI-0340/SEG_Series-28636/00000001.dcm       │ 1                 │ 53             │
│ testdata/LIDC-IDRI-0340/SEG_Series-35405/00000001.dcm       │ 1                 │ 14             │
│ testdata/LIDC-IDRI-0340/SEG_Series-14704/00000001.dcm       │ 1                 │ 55             │
│ testdata/LIDC-IDRI-0340/SEG_Series-75157/00000001.dcm       │ 1                 │ 53             │
│ testdata/PCAMPMRI-00002/SEG_Series-23.52/00000001.dcm       │ 1                 │ 30             │
│ testdata/PCAMPMRI-00002/SEG_Series-1.606/00000001.dcm       │ 1                 │ 16             │
│ testdata/PCAMPMRI-00002/SEG_Series-5.610/00000001.dcm       │ 1                 │ 30             │
│ testdata/PCAMPMRI-00002/SEG_Series-2.201/00000001.dcm       │ 1                 │ 20             │
│ testdata/PCAMPMRI-00002/SEG_Series-4.213/00000001.dcm       │ 1                 │ 16             │
│ testdata/PCAMPMRI-00002/SEG_Series-4.807/00000001.dcm       │ 1                 │ 16             │
│ testdata/ISPY2-111038/SEG_Series-94849/00000001.dcm         │ 0                 │ 64             │
│ testdata/ISPY2-111038/SEG_Series-95892/00000001.dcm         │ 0                 │ 64             │
│ testdata/ISPY2-111038/SEG_Series-24268/00000001.dcm         │ 0                 │ 64             │
│ testdata/ISPY2-111038/SEG_Series-64444/00000001.dcm         │ 0                 │ 64             │
│ testdata/R01-050/SEG_Series-51215/00000001.dcm              │ 1                 │ 291            │
│ testdata/case_0002/SEG_Series-87693/00000001.dcm            │ 1                 │ 23             │
│ testdata/case_0005/SEG_Series-33837/00000001.dcm            │ 1                 │ 11             │
│ testdata/R01-001/SEG_Series-67652/00000001.dcm              │ 1                 │ 304            │
│ testdata/R01-039/SEG_Series-61879/00000001.dcm              │ 1                 │ 125            │
│ testdata/Adrenal_Ki67_Seg_004/SEG_Series-36430/00000001.dcm │ 1                 │ 88             │
│ testdata/LIDC-IDRI-0151/SEG_Series-77386/00000001.dcm       │ 1                 │ 4              │
│ testdata/LIDC-IDRI-0151/SEG_Series-45639/00000001.dcm       │ 1                 │ 3              │
│ testdata/LIDC-IDRI-0151/SEG_Series-06499/00000001.dcm       │ 1                 │ 4              │
│ testdata/LIDC-IDRI-0151/SEG_Series-93493/00000001.dcm       │ 1                 │ 3              │
│ testdata/LIDC-IDRI-0151/SEG_Series-23735/00000001.dcm      

In [9]:
sr_files = groups["SR"]

sr_table = Table(title="SR Reference UIDs")
sr_table.add_column("SR File")
sr_table.add_column("# Ref Series UIDs")
sr_table.add_column("# Ref SOP UIDs")

for sr in sr_files:
    match sr_reference_uids(sr):
        case SR_RefSeries(ref_uid), SR_RefSOPs(ref_sops):
            sr_table.add_row(str(sr.relative_to(testpath.parent)), str(len(ref_uid)), str(len(ref_sops)))
        case SR_RefSOPs(ref_sops):
            sr_table.add_row(str(sr.relative_to(testpath.parent)), "0", str(len(ref_sops)))
        
print(sr_table)

                                         SR Reference UIDs                                         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ SR File                                                    ┃ # Ref Series UIDs ┃ # Ref SOP UIDs ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ testdata/PCAMPMRI-00001/SR_Series-9.297/00000001.dcm       │ 2                 │ 15             │
│ testdata/PCAMPMRI-00001/SR_Series-8.730/00000001.dcm       │ 2                 │ 21             │
│ testdata/PCAMPMRI-00001/SR_Series-0.565/00000001.dcm       │ 2                 │ 31             │
│ testdata/PCAMPMRI-00001/SR_Series-1.304/00000001.dcm       │ 2                 │ 21             │
│ testdata/PCAMPMRI-00001/SR_Series-6.733/00000001.dcm       │ 2                 │ 17             │
│ testdata/PCAMPMRI-00001/SR_Series-18.24/00000001.dcm       │ 2                 │ 31             │
│ testdata/OCT-01-1271/SR_Series-72387/IM-0229-0005.dcm      │ 1                 │ 5              │
│ testdata/OCT-01-1271/SR_Series-38961/IM-0230-0005.dcm      │ 1                 │ 5              │
│ testdata/OCT-01-1271/SR_Series-36213/IM-0222-0004.dcm      │ 1                 │ 4              │
│ testdata/OCT-01-0102/SR_Series-48980/IM-0044-0005.dcm      │ 1                 │ 4              │
│ testdata/OCT-01-0102/SR_Series-48980/IM-0569-0005.dcm      │ 1                 │ 4              │
│ testdata/OCT-01-0102/SR_Series-97725/IM-0031-0005.dcm      │ 1                 │ 5              │
│ testdata/OCT-01-0102/SR_Series-97725/IM-0556-0005.dcm      │ 1                 │ 5              │
│ testdata/OCT-01-0102/SR_Series-97096/IM-0557-0005.dcm      │ 1                 │ 5              │
│ testdata/OCT-01-0102/SR_Series-97096/IM-0032-0005.dcm      │ 1                 │ 5              │
│ testdata/OCT-01-0102/SR_Series-99038/IM-0043-0005.dcm      │ 1                 │ 4              │
│ testdata/OCT-01-0102/SR_Series-99038/IM-0568-0005.dcm      │ 1                 │ 4              │
│ testdata/LIDC-IDRI-0340/SR_Series-22237/00000001.dcm       │ 2                 │ 422            │
│ testdata/LIDC-IDRI-0340/SR_Series-31949/00000001.dcm       │ 2                 │ 422            │
│ testdata/LIDC-IDRI-0340/SR_Series-76739/00000001.dcm       │ 2                 │ 422            │
│ testdata/LIDC-IDRI-0340/SR_Series-66150/00000001.dcm       │ 2                 │ 422            │
│ testdata/LIDC-IDRI-0340/SR_Series-39944/00000001.dcm       │ 2                 │ 422            │
│ testdata/LIDC-IDRI-0340/SR_Series-95985/00000001.dcm       │ 2                 │ 422            │
│ testdata/LIDC-IDRI-0340/SR_Series-60793/00000001.dcm       │ 2                 │ 422            │
│ testdata/LIDC-IDRI-0340/SR_Series-01024/00000001.dcm       │ 2                 │ 422            │
│ testdata/OCT-01-0736/SR_Series-66085/IM-0222-0002.dcm      │ 1                 │ 1              │
│ testdata/OCT-01-0736/SR_Series-66085/IM-0383-0002.dcm      │ 1                 │ 1              │
│ testdata/OCT-01-0736/SR_Series-79932/IM-0213-0002.dcm      │ 1                 │ 1              │
│ testdata/OCT-01-0736/SR_Series-79932/IM-0374-0002.dcm      │ 1                 │ 1              │
│ testdata/OCT-01-0736/SR_Series-27129/IM-0214-0002.dcm      │ 1                 │ 1              │
│ testdata/OCT-01-0736/SR_Series-27129/IM-0375-0002.dcm      │ 1                 │ 1              │
│ testdata/OCT-01-0736/SR_Series-49804/IM-0384-0002.dcm      │ 1                 │ 1              │
│ testdata/OCT-01-0736/SR_Series-49804/IM-0223-0002.dcm      │ 1                 │ 1              │
│ testdata/OCT-01-1191/SR_Series-12663/IM-0299-0006.dcm      │ 2                 │ 7              │
│ testdata/OCT-01-1191/SR_Series-12663/IM-0308-0006.dcm      │ 2                 │ 7              │
│ testdata/OCT-01-1191/SR_Series-76894/IM-0309-0006.dcm      │ 2                 │ 7              │


In [20]:
from tqdm.autonotebook import tqdm

weird = [*groups["RTDOSE"], *rtplan_files, *seg_files, *sr_files, *groups["RTSTRUCT"], *groups["CT"], *groups["MR"], *groups["PT"]]
weird_parsed = []
for w in tqdm(weird):
    try:
        weird_parsed.append(parse_dicom(w))
    except Exception as e:
        print(f"Error parsing {w}: {e}")


  0%|          | 0/84519 [00:00<?, ?it/s]

  1%|          | 693/84519 [00:17<34:54, 40.02it/s]  


KeyboardInterrupt: 

In [19]:
import pandas as pd
from pandas import json_normalize

# Flatten the dictionaries and create DataFrames
dfs = [json_normalize(w) for w in weird_parsed]
df = pd.concat(dfs, ignore_index=True)
# remove the filepath column
df = df.drop(columns=["filepath"])

df

,PatientID,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,Modality,FrameOfReferenceUID,ReferencedSOPInstanceUID,RTDOSERefStructSOP,RTDOSERefPlanSOP,RTPLANRefStructSOP,ReferencedSeriesUID
0,HN-HGJ-072,1.3.6.1.4.1.14519.5.2.1.5168.2407.233545666306...,1.3.6.1.4.1.14519.5.2.1.5168.2407.269371969014...,1.3.6.1.4.1.14519.5.2.1.5168.2407.531562124805...,RTDOSE,1.3.6.1.4.1.14519.5.2.1.5168.2407.158839962192...,1.3.6.1.4.1.14519.5.2.1.5168.2407.921846643422...,1.3.6.1.4.1.14519.5.2.1.5168.2407.921846643422...,NaN,NaN,NaN
1,HN-HGJ-072,1.3.6.1.4.1.14519.5.2.1.5168.2407.756484993547...,1.3.6.1.4.1.14519.5.2.1.5168.2407.334308688250...,1.3.6.1.4.1.14519.5.2.1.5168.2407.112329825176...,RTDOSE,1.3.6.1.4.1.14519.5.2.1.5168.2407.224471774536...,1.3.6.1.4.1.14519.5.2.1.5168.2407.106404316007...,1.3.6.1.4.1.14519.5.2.1.5168.2407.106404316007...,NaN,NaN,NaN
2,VS-SEG-002,1.3.6.1.4.1.14519.5.2.1.2146429604232375654471...,1.3.6.1.4.1.14519.5.2.1.2312603692751891713589...,1.3.6.1.4.1.14519.5.2.1.1614181921545286225703...,RTDOSE,1.3.6.1.4.1.14519.5.2.1.3234962357111403548336...,1.3.6.1.4.1.14519.5.2.1.1295381050312110912875...,NaN,1.3.6.1.4.1.14519.5.2.1.1295381050312110912875...,NaN,NaN
3,VS-SEG-002,1.3.6.1.4.1.14519.5.2.1.2146429604232375654471...,1.3.6.1.4.1.14519.5.2.1.2263372411356965457223...,1.3.6.1.4.1.14519.5.2.1.1558857347201472295276...,RTDOSE,1.3.6.1.4.1.14519.5.2.1.3234962357111403548336...,1.3.6.1.4.1.14519.5.2.1.1862471466198123065560...,NaN,1.3.6.1.4.1.14519.5.2.1.1862471466198123065560...,NaN,NaN
4,VS-SEG-001,1.3.6.1.4.1.14519.5.2.1.2674248213846638137808...,1.3.6.1.4.1.14519.5.2.1.3111935591986827656438...,1.3.6.1.4.1.14519.5.2.1.1137257506625090799968...,RTDOSE,1.3.6.1.4.1.14519.5.2.1.3263657427692460239850...,1.3.6.1.4.1.14519.5.2.1.1203774555098766706658...,NaN,1.3.6.1.4.1.14519.5.2.1.1203774555098766706658...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
577,OCT-01-1303,1.3.6.1.4.1.12201.1109.13661133743850639572713...,1.2.752.243.1.1.20240903184430224.2200.65687.1,1.2.752.243.1.1.20240903184430224.2200.65687,RTSTRUCT,1.3.6.1.4.1.12201.1109.19362084419441258738557...,NaN,NaN,NaN,NaN,1.3.6.1.4.1.12201.1109.32518289154523966610386...
578,HNSCC-01-0005,1.3.6.1.4.1.14519.5.2.1.1706.8040.135679639275...,1.3.6.1.4.1.14519.5.2.1.1706.8040.458338964033...,1.3.6.1.4.1.14519.5.2.1.1706.8040.107238381189...,RTSTRUCT,,NaN,NaN,NaN,NaN,1.3.6.1.4.1.14519.5.2.1.1706.8040.218267768790...
579,HN1600,1.3.6.1.4.1.40744.29.2567103918185170854055100...,1.3.6.1.4.1.40744.29.8646651130754985590950727...,1.3.6.1.4.1.40744.29.3095632643069272941075671...,RTSTRUCT,,NaN,NaN,NaN,NaN,1.3.6.1.4.1.40744.29.4102576699412926455275059...
580,LUNG1-001,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.6468474582136099606367...,RTSTRUCT,,NaN,NaN,NaN,NaN,1.3.6.1.4.1.32722.99.99.2989917765213423750108...
